In [1]:
import sys
sys.path.append('../scripts')

In [2]:
import pandas as pd
import openai
from pathlib import Path
from evaluation import error_analysis, get_scores
import re

In [3]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path='..', version_base='1.3.1')
config = compose(config_name='experiment.yaml')

In [4]:
openai.api_key = config.api_key

In [5]:
MODEL = "gpt-4-0125-preview"
#MODEL = "gpt-3.5-turbo-1106"

In [6]:
prompt_to_use = 0
prompts = [
    "Ich werde dir im Folgenden einen Satz zeigen, welcher sogennannte Koordinationsellipsen enthält. Du sollst sie finden und auflösen. Ein Beispiel:'Dieser Unterschied zeigte sich sowohl im Stadium I/II als auch im Stadium III [REF].' Deine Antwort:'Dieser Unterschied zeigte sich sowohl im Stadium I/Stadium II als auch im Stadium III [REF].'. Ein weiteres Beispiel:'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und –ausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Deine Antwort:'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und Tumorausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Ein weiteres Beispiel:'Eine Metaanalyse von 16 randomisierten Studien mit BRAF- oder/und MEK-Inhibitoren bestätigt die Überlegenheit der Kombinationstherapie [REF].' Deine Antwort:'Eine Metaanalyse von 16 randomisierten Studien mit BRAF-Inhibtoren oder/und MEK-Inhibitoren bestätigt die Überlegenheit der Kombinationstherapie [REF].' Ein letztes Beispiel:'Das Erkrankungs- und Sterberisiko ist jedoch deutlich unterschiedlich.' Deine Antwort:'Das Erkrankungsrisiko und Sterberisiko ist jedoch deutlich unterschiedlich.' Wenn ich dir gleich mein Beispiel gebe, antworte nur mit dem korrigierten Satz und keiner Erklärung. Korrigiere bitte folgenden Satz:",
    "Ich werde dir im Folgenden einen Satz zeigen, welcher sogennannte Koordinationsellipsen enthält. Du sollst sie finden und auflösen. Ein Beispiel: 'Dieser Unterschied zeigte sich sowohl im Stadium I/II als auch im Stadium III [REF].' Deine Antwort: 'Dieser Unterschied zeigte sich sowohl im Stadium I/Stadium II als auch im Stadium III [REF].' Ein weiteres Beispiel: 'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und –ausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Deine Antwort: 'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und Tumorausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Korrigiere bitte folgenden Satz:",
    "Ich werde dir im Folgenden einen Satz zeigen. Möglicherweise enthält dieser Koordinationsellipsen. Ich möchte, dass du erstmal herausfindest, ob in dem Satz solche Koordinationsellipsen vorkommen. Danach sollst du sie auflösen. Ein Beispiel: 'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und –ausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Hier kannst du 'Tumormerkmale und -ausdehung' als Koordinationsellipse erkennen. Deine Antwort wäre nun: 'Ziel ist eine möglichst genaue Beschreibung der Tumormerkmale und Tumorausdehnung um eine optimale an das Erkrankungsstadium angepasste Therapie zu ermöglichen.' Du sollst also nichts an dem Satz ändern außer die Koordinationsellipsen zu korrigieren. Wenn ich dir gleich mein Beispiel gebe, antworte nur mit dem korrigierten Satz und keiner Erklärung. Mein Beispiel:",
    "Ich werde dir im Folgenden einen Satz zeigen, welcher sogennannte Koordinationsellipsen enthält. Du sollst sie finden und auflösen. Ein Beispiel:'Ibrutinib, ein Inhibitor der Bruton-Tyrosinkinase (BTK), ist in Deutschland als Erstlinien- und Rezidivtherapiee in der CLL zugelassen.' Deine Antwort:'Ibrutinib, ein Inhibitor der Bruton-Tyrosinkinase (BTK), ist in Deutschland als Erstlinientherapie und Rezidivtherapiee in der CLL zugelassen.' Korrigiere bitte folgenden Satz:",
    "In der deutschen Sprache gibt es sogenannte Koordinationsellipsen. Ein Beispiel für eine Koordinationsellipse ist:'Stadium I/II'. Die richtige Auflösung:'Stadium I/Stadium II'. Ein weiteres Beispiel:'Tumormerkmale und –ausdehnung'. Die richtige Auflösung:'Tumormerkmale und Tumorausdehnung'. Ein weiteres Beispiel:'BRAF- oder/und MEK-Inhibitoren'. Die richtige Auflösung:'BRAF-Inhibtoren oder/und MEK-Inhibitoren'. Ein letztes Beispiel:'Erkrankungs-, Verhinderungs- und Sterberisiko' Die richtige Auflösung:'Erkrankungsrisiko und Sterberisiko'. Ich werde dir im Folgenden einen Satz zeigen, welcher eine oder mehrere dieser Koordinationsellipsen enthält. Ich möchte, dass du sie findest und auflöst. Ein Beispiel:'Das Erkrankungs- und Sterberisiko ist jedoch deutlich unterschiedlich [REF].' Deine Antwort:'Das Erkrankungsrisiko und Sterberisiko ist jedoch deutlich unterschiedlich [REF].' Korrigiere bitte folgenden Satz:"
]

In [7]:
ellipses = pd.read_csv(Path('..') / config.data.cnf_tsv_path, sep='\t')
not_ellipses = pd.read_csv(Path('..') / config.data.controls_tsv_path, sep='\t')

not_ellipses['full_resolution'] = not_ellipses.raw_sentence
ellipses = ellipses[ellipses.split == 'test']
not_ellipses = not_ellipses[not_ellipses.split == 'test']

data = pd.concat([ellipses[['file', 'sentence_id', 'raw_sentence', 'full_resolution']], not_ellipses[['file', 'sentence_id', 'raw_sentence', 'full_resolution']]]).sort_values(by='file').reset_index(drop=True)

In [8]:
test_data = data#data.sample(20, random_state=1)

In [9]:
from retry import retry

@retry(tries=3, delay=2)
def get_openai_response(prompt, example):
    return openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": prompt + example}
        ],
        temperature=0,
        max_tokens=100,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )["choices"][0]["message"]["content"]

In [18]:
len(gpt_predictions)

1061

In [17]:
gpt_predictions += new_gpt_predictions

In [19]:
from tqdm.auto import tqdm

new_gpt_predictions = []
i = 1
for example, resolution in tqdm(zip(test_data.raw_sentence, test_data.full_resolution), total=len(test_data)):
    if i > len(gpt_predictions):
        new_gpt_predictions.append(get_openai_response(prompts[prompt_to_use], example))
    i += 1

  0%|          | 0/1159 [00:00<?, ?it/s]

RateLimitError: Rate limit reached for gpt-4-turbo-preview in organization org-bvToJJDixUHlY7V0ejxgC0mo on tokens per day (TPD): Limit 500000, Used 499844, Requested 461. Please try again in 52.704s. Visit https://platform.openai.com/account/rate-limits to learn more.

In [20]:
import pickle

with open(f'../outputs/predictions_zeroshot_{MODEL}.pkl', 'wb') as fh:
    pickle.dump(gpt_predictions, fh)

In [21]:
import pickle

with open(f'../outputs/predictions_zeroshot_{MODEL}.pkl', 'rb') as fh: 
    predictions = pickle.load(fh)

In [22]:
for i, prediction in enumerate(predictions):
    predictions[i] = prediction.replace('\n', '')
    predictions[i] = re.sub(r'.*Antwort:\s?', '', predictions[i])
    if predictions[i][0] == "'":
        predictions[i] = predictions[i][1:-1]

In [25]:
n = len(predictions)

In [26]:
ea = error_analysis(predictions, test_data.full_resolution[0:n], test_data.raw_sentence[0:n])
print(get_scores(ea, "eval"))

{'eval/exact_match': 0.30254476908576816, 'eval/gleu': 0.8140001010594398, 'eval/edit_distance_rel': 0.4430652234220432}


In [36]:
fac = 1159 / ea.error_type.value_counts().sum()
(ea.error_type.value_counts() * fac).round()

fp         480.0
tp         220.0
complex    134.0
tn         131.0
insert      69.0
delete      67.0
replace     55.0
fn           4.0
Name: error_type, dtype: float64

In [37]:
from notebook_util import show_errors

In [38]:
ea['file'] = 'egal'
ea['sentence_id'] = 0

In [39]:
show_errors(ea)

0;egal;0

__Input:__

Hauptrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabak- oder Alkoholabusus, wesentlich seltener auch andere Faktoren.

__Prediction (correct):__

Hauptrisikofaktoren für das Auftreten eines Mundhöhlenkarzinoms sind chronischer Tabakabusus oder Alkoholabusus, wesentlich seltener auch andere Faktoren.

...

__Error type:__ fp

1060;egal;0

__Input:__

In der Netzwerkmetaanalyse, die im November 2015 für die britische NICE-Leitlinie „Menopause – Clinical Guideline, Methods, Evidence and Recommendations“ [REF], [REF] durchgeführt worden ist, wurde ein signifikanter Therapieeffekt von CR versus Placebo in der Reduktion von Hitzewallungen (MR 0.4; 95% KI 0.17-0.9) festgestellt (S. 94 und Appendix k, S. 241).

__Error type:__ fp

In [ ]:
test_data.loc[170].raw_sentence

In [ ]:
test_data.loc[170].full_resolution

In [ ]:
final.loc[170].prediction

In [ ]:
final.loc[170].resolution

In [ ]:
final = pd.DataFrame(data={'resolution': test_data.full_resolution, 'prediction': predictions})
final[final.prediction!=final.resolution]